https://github.com/openai/openai-agents-python/blob/main/examples/basic/agent_lifecycle_example.py 

two agents
each has its own tools 

1. starter agents: will process the initial user request, will use tools as needed and will handoff to other agents if required
2. multiply agent: knows how to perform arithmetic operations (can we configure a handoff)

example also keeps tracks of agent life cycle events 


In [1]:
from dotenv import load_dotenv
from agents import (

    Agent,
    Runner,
    RunConfig,
    trace,
)

from pydantic import BaseModel, Field
load_dotenv()

True

In [2]:
from agents import Agent, RawResponsesStreamEvent, Runner, TResponseInputItem, trace
from helpers.trace_util import get_trace_url
from agents.extensions.visualization import draw_graph

In [3]:
from helpers.model_client import (
    get_openai_client,
    get_github_model_provider
)

model_name = "gpt-4o-mini"

GITHUB_MODEL_PROVIDER = get_github_model_provider(
    client = get_openai_client(),
    model = model_name
)

In [4]:
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
from agents import handoff, RunContextWrapper


class Joke(BaseModel):
    joke: str = Field(description="The joke text")
    punchline: str = Field(description="The punchline of the joke")
    language: str = Field(description="The language of the joke")
    spanish_translation: str = Field(description="The joke translated to Spanish")
    french_translation: str = Field(description="The joke translated to French")


# class Translation(BaseModel):
#     content: str = Field(description="Content")
#     language: str = Field(description="The language of the translation")

# async def on_handoff1(ctx: RunContextWrapper[None], input_data: Translation):
#     print(f"agent {ctx} called with: {input_data.content}")

dad_jokes_agent = Agent(
    name="dad_jokes_generator_agent",
    instructions=(
        f"{RECOMMENDED_PROMPT_PREFIX} \n"
        "You are an agent that specialises in generating jokes, dad jokes specifically. "
        "You only respond with dad jokes."
        "You do not answer any other questions. "
        "Return the job to the orchestrator agent once you produced the joke "
        "always deliver a punchline. "
        " ## IMPORTANT ##"
        "Always respond in English, even if the user asks in a different language. "
        "You are not allowed to use any other language than English. "
        "You are not allowed to end the conversation, must handoff to an approriate agent."
    ),
    handoff_description=(
        "An agent capable of generating jokes based on user input "
        "you only generate content in English, regardless of the input language, or user requested language"
    ),
    model=model_name,
    output_type=Joke,
)


spanish_agent = Agent[Joke](
    name="spanish_agent",
    instructions= (
        f"{RECOMMENDED_PROMPT_PREFIX} \n"
        "You translate the input message to Spanish"
        "You only translate, you dont generate new content"
        "handoff the job back to the orchestrator once you produced the translation"
        "if you cannot answer, transfer back to the orchestrator"
    ),
    handoff_description="An agent capable of translating messages to Spanish",
    model=model_name,
    output_type=Joke,

)



french_agent = Agent[Joke](
    name="french_agent",
    instructions= (
        f"{RECOMMENDED_PROMPT_PREFIX} \n"
        "You translate the user's message to French"
        "you only translate, you dont generate new content"
        "handoff the job back to the orchestrator once you produced the translation"
        "if you cannot answer, transfer back to the orchestrator"
    ),
    handoff_description="An agent capable of translating messages to French",
    model=model_name,
    output_type=Joke,
)





In [5]:

def handoff_instructions_msg(lang: str):
    return {"role": "user", "content": f"Translate the following text to {lang}"}

In [6]:
input_list = [
    {"role": "user", "content": "Tell me a dad joke"},
]


# Implement Handoffs

In this example, we will implement the handoffs using the SDK handoff functionality.

In [7]:

user_response_agent = Agent[Joke](
    name="user_response_agent",
    instructions=(
        f"{RECOMMENDED_PROMPT_PREFIX} \n"
        "You are an agent that generates a response to the user's message."
        "your response should include summary of steps taken to produce the final output  in bullet points"
        "present this execution summary as a list of steps taken to produce the final output."
        "then in the response back to the user, include the final output from each step, and the very last final output"
    ),
    handoff_description="An agent capable of generating final user responses",
    model=model_name,
)



In [8]:

orchestrator_agent = Agent[Joke](
    name="orchestrator_agent",
    model=model_name,
    tool_use_behavior="run_llm_again",
    instructions=(
        f"{RECOMMENDED_PROMPT_PREFIX} \n"
        "You are an orchestrator agent. "
        "You handoff the task to the appropriate task agent."
        "The task agents are: "
        f"{dad_jokes_agent.name}, {spanish_agent.name}, {french_agent.name}, {user_response_agent.name}."
        "You may need to breakdown the task and manage multiple handoffs as needed."
        "If asked for multiple translations, handoff to multiple agents."
        "You only speak English. Do not translate the user's message yourself."
        "Use the responses from other agents to generate the final response."
        "You are not allowed to end the conversation, must handoff to an approriate agent."
    ),
    handoff_description=(
        "An agent that orchestrates the task by indentifing an "
        "executation plan based on available agents and tools "
        "and then  handing off to other agents."
        ),
    handoffs=[dad_jokes_agent,spanish_agent, french_agent,user_response_agent],
)


In [9]:


dad_jokes_agent.handoffs = [orchestrator_agent]
spanish_agent.handoffs = [orchestrator_agent]
french_agent.handoffs = [orchestrator_agent]



In [10]:



try:
    gv = draw_graph(
        orchestrator_agent,
        filename="./viz/07_agent_handoffs_orchestrator_agent_v2.gv",

    )

except Exception as e:
    print(f"Error: {e}")



Error: maximum recursion depth exceeded while calling a Python object


# Implementing Agent Handoffs with the Agents SDK

![handoff flow](./viz/07_agent_handoffs_orchestrator_agent.svg)

This visualization shows the agent flow we've created, where the orchestrator agent directs requests to the appropriate specialized agents. The dad jokes generator produces jokes, which can then be handed off to language agents (Spanish or French) for translation, and finally to the user response agent to format the final response.

In [11]:
from agents import Agent, RawResponsesStreamEvent, Runner, TResponseInputItem, trace

msg = input ("Enter your message: ") or "Hello, tell me a joke in spanish"

inputs: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

result = None
try:
    with trace(workflow_name="handoffs", group_id="mdsi") as tr:
        print(f"Trace URL: {get_trace_url(tr)}")
        result = await Runner.run(
            starting_agent=orchestrator_agent,
            input=inputs,
            run_config=RunConfig(model=model_name, model_provider=GITHUB_MODEL_PROVIDER),
        )

        print(result.final_output)
        print(result.last_agent)
except Exception as e:
    print(f"Error: {e}")







Trace URL: https://platform.openai.com/traces/trace?trace_id=trace_ab3fa25b89fb4bb2921fb1138829ef38
joke='Why did the scarecrow win an award?' punchline='Because he was outstanding in his field!' language='English' spanish_translation='' french_translation=''
Agent(name='dad_jokes_generator_agent', instructions='# System context\nYou are part of a multi-agent system called the Agents SDK, designed to make agent coordination and execution easy. Agents uses two primary abstraction: **Agents** and **Handoffs**. An agent encompasses instructions and tools and can hand off a conversation to another agent when appropriate. Handoffs are achieved by calling a handoff function, generally named `transfer_to_<agent_name>`. Transfers between agents are handled seamlessly in the background; do not mention or draw attention to these transfers in your conversation with the user.\n \nYou are an agent that specialises in generating jokes, dad jokes specifically. You only respond with dad jokes.You do n

In [ ]:
print(result)

print(get_trace_url(tr))

In [ ]:
result.raw_responses

In [ ]:
from helpers.trace_util import display_agent_execution_steps


display_agent_execution_steps(result)

In [ ]:
tracing_url = f"https://platform.openai.com/traces/trace?trace_id={tr.trace_id}"

print(f"Tracing URL: {tracing_url}")

## Visualise the Agent

In [ ]:
from agents.extensions.visualization import draw_graph

try:
    draw_graph(orchestrator_agent, filename="viz/07_agent_handoffs.gv")
except Exception as e:
    print(f"An error occurred: {e}")

# another explains

In [ ]:

french_agent = Agent(
    name="french_agent",
    instructions="You only speak French",
)

spanish_agent = Agent(
    name="spanish_agent",
    instructions="You only speak Spanish",
)

english_agent = Agent(
    name="english_agent",
    instructions="You only speak English",
)

triage_agent = Agent(
    name="triage_agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[french_agent, spanish_agent, english_agent],
)


In [ ]:
from openai.types.responses import ResponseContentPartDoneEvent, ResponseTextDeltaEvent

from agents import Agent, RawResponsesStreamEvent, Runner, TResponseInputItem, trace
import uuid

conversation_id = str(uuid.uuid4().hex[:16])

msg = input("Hi! We speak French, Spanish and English. How can I help? ")
agent = triage_agent
inputs: list[TResponseInputItem] = []
inputs.append({"content": msg, "role": "user"})

while True:
    # Each conversation turn is a single trace. Normally, each input from the user would be an
    # API request to your app, and you can wrap the request in a trace()




    with trace("Routing example", group_id=conversation_id):
        result = Runner.run_streamed(
            agent,
            input=inputs,
            run_config=RunConfig(model=model_name, model_provider=GITHUB_MODEL_PROVIDER),
        )
        async for event in result.stream_events():
            if not isinstance(event, RawResponsesStreamEvent):
                continue
            data = event.data
            if isinstance(data, ResponseTextDeltaEvent):
                print(data.delta, end="", flush=True)
            elif isinstance(data, ResponseContentPartDoneEvent):
                print("\n")

    inputs = result.to_input_list()
    print("\n")


    agent = result.current_agent
    user_msg = input("Enter a message: ")
    if user_msg in ("exit", "quit", ""):
        break
    inputs.append({"content": user_msg, "role": "user"})

In [ ]:


try:
    draw_graph(orchestrator_agent, filename="viz/07_agent_handoffs.gv")
except Exception as e:
    print(f"An error occurred: {e}")